In [1]:
import job_scraper
from job_scraper import find_jobs_from

## Extracting jobs from Indeed.com or Hinge Health website

In [ ]:
desired_characs = ['titles', 'companies', 'links', 'date_listed']

In [ ]:
find_jobs_from('Indeed', 'data analyst', 'california', desired_characs)

In [2]:
desired_characs = ['titles','links']

In [6]:
##compare with new function:
find_jobs_from('Hinge', '', '', desired_characs, filename = 'results2.xls')

10 new job postings retrieved from Hinge. Stored in results2.xls.


## Extracting jobs from Hinge Health

In [ ]:
def load_hinge_jobs_div(commitment='Full-time', team='Engineering'):
    getVars = {'commitment': commitment, 'team': team}
    url = ('https://jobs.lever.co/hingehealth?' + urllib.parse.urlencode(getVars))
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    # job_soup = soup.find(id="resultsCol")
    # job_soup = soup.find(data-qa="posting-name")
#     job_soup = soup.find_all('h5') ##this returns all the job titles/names, but try to get the LINKS posted as well
#     job_soup = soup.find_all('a', class_="posting-title")##trying to get the link for job
    job_soup = soup.find(class_="postings-group")
    return job_soup

In [ ]:
test_soup = load_hinge_jobs_div(commitment='Full-time', team='Engineering') ## find_all funct returns a <class 'bs4.element.ResultSet'>

In [ ]:
##extract info from soup
def extract_job_information_hinge(job_soup, desired_characs):
    job_elems = job_soup.find_all('a', class_='posting-title')

    cols = []
    extracted_info = []

    if 'titles' in desired_characs:
        titles = []
        cols.append('titles')
        for job_elem in job_elems:
            titles.append(extract_job_title_hinge(job_elem))
        extracted_info.append(titles)

#     if 'companies' in desired_characs:
#         companies = []
#         cols.append('companies')
#         for job_elem in job_elems:
#             companies.append(extract_company_indeed(job_elem))
#         extracted_info.append(companies)

    if 'links' in desired_characs:
        links = []
        cols.append('links')
        for job_elem in job_elems:
            links.append(extract_link_hinge(job_elem))
        extracted_info.append(links)

#     if 'date_listed' in desired_characs:
#         dates = []
#         cols.append('date_listed')
#         for job_elem in job_elems:
#             dates.append(extract_date_indeed(job_elem))
#         extracted_info.append(dates)

    jobs_list = {}

    for j in range(len(cols)):
        jobs_list[cols[j]] = extracted_info[j]

    num_listings = len(extracted_info[0])

    return jobs_list, num_listings

def extract_job_title_hinge(job_elem):
    title_elem = job_elem.find('h5')
    title = title_elem.text.strip()
    return title

def extract_link_hinge(job_elem):
    link = job_elem['href'] ##basically found the element in soup that gave you link. its tag = a, value = href
    return link


In [ ]:
# job_elem
# job_elem['href']

In [ ]:
test_soup.find('a')['href'] ##found element wanted

In [ ]:
# now test the whole extract_job_information_hinge function
desired_characs = ['titles', 'links']
jobs_list, num_listings = extract_job_information_hinge(load_hinge_jobs_div(),desired_characs)

In [5]:
##compare results to the indeed query/function
load_hinge_jobs_div()
type(load_hinge_jobs_div())

In [ ]:
def load_indeed_jobs_div(job_title, location):
    getVars = {'q': job_title, 'l': location, 'fromage': 'last', 'sort': 'date'}
    url = ('https://www.indeed.com/jobs?' + urllib.parse.urlencode(getVars))
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    job_soup = soup.find(id="resultsCol")
    return job_soup

In [ ]:
job_title="data scientist"
location = "california"
load_indeed_jobs_div(job_title,location) ## returns a <class 'bs4.element.Tag'> from "find" function
# type(load_indeed_jobs_div(job_title,location))

## Extracting jobs from CWjobs.co.uk (using Selenium)

In [ ]:
find_jobs_from('CWjobs', 'data scientist', 'london', desired_characs)